In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    nhl_df1 = nfl_df[(nfl_df['year']==2018) & (nfl_df['team']!=nfl_df['L'])]
    nhl_df1['team'] = nhl_df1['team'].str.replace(r'\*','') 
    nhl_df1 = nhl_df1[['team','W','L']]
    data_types_dict = {'team': str,'W' : int, 'L': int}
    nhl_df1 = nhl_df1.astype(data_types_dict)
    nhl_df1['ratio'] = nhl_df1['W']/(nhl_df1['W']+nhl_df1['L'])
    nhl_df1['team'] = nhl_df1['team'].str.replace(r'[\*]',"")
    nhl_df1['team'] = nhl_df1['team'].str.replace(r'\(\d*\)',"")
    nhl_df1['team'] = nhl_df1['team'].str.replace(r'[\xa0]',"")
    nhl_df1['team'] = nhl_df1['team'].str.replace('\ Sox','Sox')
    nhl_df1['team'] = nhl_df1['team'].str.replace('[\w.]* ','')
    nhl_df1['team'] = nhl_df1['team'].str.replace('+','')


    print(nhl_df1)
    #print(nhl_df1.head(100))
    team = cities['NFL'].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population (2016 est.)[8]': int})
    team['team']=team['team'].str.replace('\ Sox','Sox')
    team['team']=team['team'].str.replace('[\w.]*\ ','')
    print(team)

    join = pd.merge(team, nhl_df1,how='inner', on='team')
    
    join1 = join.groupby('Metropolitan area').agg({'ratio':np.nanmean,'Population (2016 est.)[8]':np.nanmean})
    population_by_region = join1['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = join1['ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
     

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()